In [8]:
# Tottenham,Newcastle
# Aston Villa,Leicester
# Bournemouth,Everton
# Crystal Palace,Chelsea
# Manchester City,West Ham
# Southampton,Brentford
# Brighton,Arsenal
# Fulham,Ipswich
# Liverpool,Manchester United
# Wolves,Nottingham Forest

In [ ]:
import requests
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [201]:

fixtures = requests.get('https://fantasy.premierleague.com/api/fixtures/?event=22').json()
boots_ = requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').json()

fixture_dets =  [{'team_a': fix['team_a'], 'team_h': fix['team_h'], 'h_fdr': fix['team_h_difficulty'], 'a_fdr': fix['team_a_difficulty'] } for fix in fixtures]
fix_df = pd.DataFrame(fixture_dets)

team_ids = [{'team_code': team['code'], 'team_name': team['name'], 'team_id': team['id']} for team in boots_['teams']]
id_df = pd.DataFrame(team_ids)

def add_team_name(row):
    away = row['team_a']
    home = row['team_h']

    home_team_row = id_df[id_df['team_id'] == home]['team_name'].values
    away_team_row = id_df[id_df['team_id'] == away]['team_name'].values

    team_home = home_team_row[0] if home_team_row.size > 0 else 'None'
    team_away = away_team_row[0] if away_team_row.size > 0 else 'None'

    if(team_home == 'Man Utd'):
        team_home = 'Manchester United'
    elif(team_away == 'Man Utd'):
        team_away = 'Manchester United'

    if(team_home == 'Man City'):
        team_home = 'Manchester City'
    elif(team_away == 'Man City'):
        team_away = 'Manchester City'

    if(team_home == 'Wolves'):
        team_home = 'Wolverhampton Wanderers'
    elif(team_away == 'Wolves'):
        team_away = 'Wolverhampton Wanderers'

    if(team_home == "Nott'm Forest"):
        team_home = "Nottingham Forest"
    elif(team_away == "Nott'm Forest"):
        team_away = "Nottingham Forest"

    if(team_home == "Spurs"):
        team_home = "Tottenham"
    elif(team_away == "Spurs"):
        team_away = "Tottenham"

    if(team_home == "Newcastle"):
        team_home = "Newcastle United"
    elif(team_away == "Newcastle"):
        team_away = "Newcastle United"


    return pd.Series([team_home, team_away])


fix_df[['team_home', 'team_away']] = fix_df.apply(add_team_name, axis=1)

fix_df.to_csv('./fpl_team_ids.csv', index=False)


In [202]:
pd.read_csv('./fpl_team_ids.csv')

,team_a,team_h,h_fdr,a_fdr,team_home,team_away
0,3,15,3,4,Newcastle United,Bournemouth
1,12,4,5,3,Brentford,Liverpool
2,9,11,3,2,Leicester,Fulham
3,7,19,3,2,West Ham,Crystal Palace
4,2,1,3,4,Arsenal,Aston Villa
5,18,8,3,3,Everton,Tottenham
6,5,14,3,3,Manchester United,Brighton
7,17,16,2,4,Nottingham Forest,Southampton
8,13,10,4,2,Ipswich,Manchester City
9,20,6,2,4,Chelsea,Wolverhampton Wanderers


In [59]:
premLeague = "https://sports.williamhill.com/betting/en-gb/football/competitions/OB_TY295/English-Premier-League/matches/OB_MGMB/Match-Betting"

driver = webdriver.Firefox()
driver.get(premLeague)
matches = driver.find_elements(By.CSS_SELECTOR, "article.sp-o-market--default")[0:10]
details = pd.DataFrame({'h_team':[], 'a_team':[], 'WHH':[], 'WHD':[], 'WHA':[]})

# Get links for the matches
# [[match, link]]
for match in matches:
    # matches_links.append([match.text, match.get_attribute('href')])
    # Extract teams
    teams = match.find_element(By.CSS_SELECTOR, 'main.sp-o-market__title span').text
    odds_els = match.find_elements(By.CSS_SELECTOR, 'section.sp-o-market__buttons .sp-betbutton > span')

    # Extract odds
    odds = [round(int(btn.text.split('/')[0]) / int(btn.text.split('/')[1]) +1, 2)   for btn in match.find_elements(By.CSS_SELECTOR, 'section.sp-o-market__buttons .sp-betbutton > span')]

    odds = []
    for btn in odds_els:
        if btn.text == 'EVS':
            odds.push(round(2.0, 2))
        else:
            odds.append(round(int(btn.text.split('/')[0]) / int(btn.text.split('/')[1]) +1, 2))
    h_odds = 1/odds[0]
    d_odds = 1/odds[1]
    a_odds = 1/odds[2]

    sum_odd_probs = h_odds + d_odds + a_odds
    match_details = {
        'h_team': [teams.split(' v ')[0]],
        'a_team': [teams.split(' v ')[1]],
        'WHH': [round(h_odds/sum_odd_probs, 3)],
        'WHD': [round(d_odds/sum_odd_probs, 3)],
        'WHA': [round(a_odds/sum_odd_probs, 3)]
        }
    match_dets_df = pd.DataFrame(match_details)
    details = pd.concat([details, match_dets_df], ignore_index=True)
    # print(odds)
    # print({'Home': teams.split(' v ')[0], 'Away': teams.split(' v ')[1], 'WHH': odds[0], 'WHD': odds[1], 'WHA': odds[2]  })


driver.close()


details

,h_team,a_team,WHH,WHD,WHA
0,Newcastle,Bournemouth,0.576,0.226,0.198
1,Brentford,Liverpool,0.162,0.187,0.651
2,Leicester,Fulham,0.241,0.251,0.508
3,West Ham,Crystal Palace,0.358,0.284,0.358
4,Arsenal,Aston Villa,0.634,0.220,0.146
5,Everton,Tottenham,0.340,0.264,0.396
6,Man Utd,Brighton,0.456,0.260,0.284
7,Nottingham Forest,Southampton,0.671,0.204,0.125
8,Ipswich,Man City,0.117,0.188,0.695
9,Chelsea,Wolves,0.702,0.172,0.126


In [60]:

# Manchester City,
# Manchester United
def update_name(row):
    if(row['h_team'] == 'Man Utd'):
        row['h_team'] = 'Manchester United'
    elif(row['a_team'] == 'Man Utd'):
        row['a_team'] = 'Manchester United'

    if(row['h_team'] == 'Man City'):
        row['h_team'] = 'Manchester City'
    elif(row['a_team'] == 'Man City'):
        row['a_team'] = 'Manchester City'

    if(row['h_team'] == 'Wolves'):
        row['h_team'] = 'Wolverhampton Wanderers'
    elif(row['a_team'] == 'Wolves'):
        row['a_team'] = 'Wolverhampton Wanderers'

    return row


details_ = details.apply(update_name, axis=1)
details_

Man Utd Brighton


,h_team,a_team,WHH,WHD,WHA
0,Newcastle,Bournemouth,0.576,0.226,0.198
1,Brentford,Liverpool,0.162,0.187,0.651
2,Leicester,Fulham,0.241,0.251,0.508
3,West Ham,Crystal Palace,0.358,0.284,0.358
4,Arsenal,Aston Villa,0.634,0.220,0.146
5,Everton,Tottenham,0.340,0.264,0.396
6,Manchester United,Brighton,0.456,0.260,0.284
7,Nottingham Forest,Southampton,0.671,0.204,0.125
8,Ipswich,Manchester City,0.117,0.188,0.695
9,Chelsea,Wolves,0.702,0.172,0.126


In [61]:

details_.to_csv('./odds_22.csv', index=False)


In [200]:
odds_

,Unnamed: 0,h_team,a_team,WHH,WHD,WHA
0,0,Newcastle United,Bournemouth,0.576,0.226,0.198
1,1,Brentford,Liverpool,0.162,0.187,0.651
2,2,Leicester,Fulham,0.241,0.251,0.508
3,3,West Ham,Crystal Palace,0.358,0.284,0.358
4,4,Arsenal,Aston Villa,0.634,0.220,0.146
5,5,Everton,Tottenham,0.340,0.264,0.396
6,6,Manchester United,Brighton,0.456,0.260,0.284
7,7,Nottingham Forest,Southampton,0.671,0.204,0.125
8,8,Ipswich,Manchester City,0.117,0.188,0.695
9,9,Chelsea,Wolverhampton Wanderers,0.702,0.172,0.126


In [197]:
teams

,team_a,team_h,h_fdr,a_fdr,team_home,team_away
0,3,15,3,4,Newcastle,Bournemouth
1,12,4,5,3,Brentford,Liverpool
2,9,11,3,2,Leicester,Fulham
3,7,19,3,2,West Ham,Crystal Palace
4,2,1,3,4,Arsenal,Aston Villa
5,18,8,3,3,Everton,Tottenham
6,5,14,3,3,Manchester United,Brighton
7,17,16,2,4,Nottingham Forest,Southampton
8,13,10,4,2,Ipswich,Manchester City
9,20,6,2,4,Chelsea,Wolverhampton Wanderers


In [207]:
# Add fdr
odds_ = pd.read_csv('./odds_22.csv')
teams = pd.read_csv('./fpl_team_ids.csv')

def add_fdr(row):
    team_row = teams[teams['team_home'] == row['h_team']]
    h_fdr = team_row['h_fdr'].values[0] if team_row['h_fdr'].values.size > 0 else 'None'
    a_fdr = team_row['a_fdr'].values[0] if team_row['a_fdr'].values.size > 0 else 'None'

    return pd.Series([h_fdr, a_fdr])

odds_[['h_fdr', 'a_fdr']] = odds_.apply(add_fdr, axis=1)
odds_.to_csv('./odds_22.csv', index=False)

In [208]:
pd.read_csv('./odds_22.csv')

,Unnamed: 0,h_team,a_team,WHH,WHD,WHA,h_fdr,a_fdr
0,0,Newcastle United,Bournemouth,0.576,0.226,0.198,3,4
1,1,Brentford,Liverpool,0.162,0.187,0.651,5,3
2,2,Leicester,Fulham,0.241,0.251,0.508,3,2
3,3,West Ham,Crystal Palace,0.358,0.284,0.358,3,2
4,4,Arsenal,Aston Villa,0.634,0.220,0.146,3,4
5,5,Everton,Tottenham,0.340,0.264,0.396,3,3
6,6,Manchester United,Brighton,0.456,0.260,0.284,3,3
7,7,Nottingham Forest,Southampton,0.671,0.204,0.125,2,4
8,8,Ipswich,Manchester City,0.117,0.188,0.695,4,2
9,9,Chelsea,Wolverhampton Wanderers,0.702,0.172,0.126,2,4
